In [12]:
import os
import ee
import numpy as np
from geeml.extract import extractor
import pandas as pd
# import geemap
# Authenticate GEE
# ee.Authenticate()
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/explore/nobackup/people/spotter5/cnn_mapping/gee-serdp-upload-7cd81da3dc69.json"

service_account = 'gee-serdp-upload@appspot.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, "/explore/nobackup/people/spotter5/cnn_mapping/gee-serdp-upload-7cd81da3dc69.json")
ee.Initialize(credentials)
# Initialize GEE with high-volume end-point
# ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
ee.Initialize()


In [13]:
import geemap
import os
from google.cloud import storage
from google.cloud import client


In [14]:
os.environ["GCLOUD_PROJECT"] = "gee-serdp-upload"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/explore/nobackup/people/spotter5/cnn_mapping/gee-serdp-upload-7cd81da3dc69.json"
storage_client = storage.Client.from_service_account_json("/explore/nobackup/people/spotter5/cnn_mapping/gee-serdp-upload-7cd81da3dc69.json")

os.environ["GCLOUD_PROJECT"] = "gee-serdp-upload"
storage_client = storage.Client()
# bucket_name = 'smp-scratch/mtbs_1985'
bucket_name = 'smp-scratch'

bucket = storage_client.bucket(bucket_name)

Import assetts of interest

In [15]:
geometry = ee.FeatureCollection('users/spotter/fire_cnn/raw/eurasia') #area of interest
mod1 = ee.ImageCollection("MODIS/061/MOD10A1") #active fire
mod2 = ee.ImageCollection("MODIS/061/MYD10A1") #active fire
fire_cci = ee.ImageCollection("ESA/CCI/FireCCI/5_1") #active fire
mod_burn = ee.ImageCollection("MODIS/061/MCD64A1") #mcd64a1
sent_2A = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") #sentinel 2
s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY') #cloud masking for sentinel
lfdb = ee.FeatureCollection("users/spotter/fire_cnn/raw/nbac_1985") #nbac_fire_polygons, this can be any polygon shapefile, final version would be nbac and mtbs
input_grid = ee.FeatureCollection('users/spotter/fire_cnn/raw/eurasia_dnbr_grid_clip') #grid to loop through to get around memory errors


Cloud masking Sentinel 2

In [16]:

#probability of clouds
MAX_CLOUD_PROBABILITY = 50

def sent_maskcloud(image):


    image = image.select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12'], ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7'])# rename bands to match landsat

    image =  image.toShort()

    clouds = ee.Image(image.get('cloud_mask')).select('probability')

    isNotCloud = clouds.lt(MAX_CLOUD_PROBABILITY)

    image = image.updateMask(isNotCloud)

    #reproject 30m but remember b1, b2 and b3 are 10 and the rest are 20
    image1 = image.select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4'])
    image2 = image.select(['SR_B5', 'SR_B7'])


    image1 = image1.reproject(
    crs = image1.projection().crs(),
    scale = 30) #resample for landsat


    image2 = image2.reproject(
    crs = image2.projection().crs(),
    scale = 30) #resample for landsat

    image = image1.addBands(image2)

    return image

#Join S2 SR with cloud probability dataset to add cloud mask.
s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(

  primary=sent_2A,
  secondary=s2Clouds,
  condition=ee.Filter.equals(leftField='system:index', rightField='system:index'))

#apply cloud masking
sent_2A = ee.ImageCollection(s2SrWithCloudMask).map(sent_maskcloud)

Correct landsat scale factor and sentinel scale factor

In [17]:
def mask(image):
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(8).eq(0).And(qa.bitwiseAnd(10).eq(0)).And(qa.bitwiseAnd(32).eq(0))
    return(image.updateMask(mask))

def land_scale(image):

    return(image.multiply(0.0000275).add(-0.2))

def sent_scale(image):
    return(image.multiply(0.0001))

Logan Coefficients to apply

In [18]:
import pandas as pd
coeffs = pd.read_csv("/explore/nobackup/people/spotter5/cnn_mapping/raw_files/boreal_xcal_regression_coefficients.csv").fillna(0)
#l5
def landsat_correct(sat, bands):

    """argument 1 is which sattelite, LANDASAT_5 or LANDSAT_8
    argument 2 is bands of interest.  Bands must be in same order as EE,
    
    regression is of form,
    L7 = B0 + (B1 * L5/8) + (B2 * L^2) + (B3 * L^3)
    """

    #bands of interest in order of interest
    l5 = coeffs[(coeffs['satellite'] == sat) & (coeffs['band.or.si'] .isin (bands))] 

    #arrange the band or si column
    l5['band.or.si']=pd.Categorical(l5['band.or.si'],categories=bands)
    l5=l5.sort_values('band.or.si')

    b0 = l5['B0'].values.tolist()
    b1 = l5['B1'].values.tolist()
    b2 = l5['B2'].values.tolist()
    b3 = l5['B3'].values.tolist()

    return (b0, b1, b2, b3)

#get the corrections, each output is a list at one of the four locations
l8_corr = landsat_correct(sat = 'LANDSAT_8', bands = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'nbr', 'ndvi', 'ndii'])
l5_corr = landsat_correct(sat = 'LANDSAT_5', bands = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'nbr', 'ndvi', 'ndii'])




/tmp/ipykernel_302470/2309367526.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  l5['band.or.si']=pd.Categorical(l5['band.or.si'],categories=bands)
/tmp/ipykernel_302470/2309367526.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  l5['band.or.si']=pd.Categorical(l5['band.or.si'],categories=bands)


I might need to convert to float

In [19]:
def to_float(image):

    b1 = image.select('SR_B1').cast({'SR_B1':'float'}) #0
    b2 = image.select('SR_B2').cast({'SR_B2':'float'}) #1
    b3 = image.select('SR_B3').cast({'SR_B3':'float'}) #2
    b4 = image.select('SR_B4').cast({'SR_B4':'float'}) #3
    b5 = image.select('SR_B5').cast({'SR_B5':'float'}) #4
    b6 = image.select('SR_B7').cast({'SR_B7':'float'}) #5

    image = b1.addBands(b2).addBands(b3).addBands(b4).addBands(b5).addBands(b6)

    return image

def filter_cl(image):
    cl = image.select('ConfidenceLevel')
    image = image.updateMask(cl.gte(50)).select('BurnDate')
    return image

def mod_act_map(image):
    image = image.addBands(ee.Image(ee.Date(image.get('system:time_start')).getRelative('day','year').add(1)).clamp(1,366)).updateMask(image.select('FireMask').gte(7))
    return image.updateMask(image.select('constant').gt(60));


Function to return the pre_fire and post_fire landsat and sentinel 2 data

In [20]:
def get_pre_post(pre_start, pre_end, post_start, post_end, geometry):

    """parameters are:
    pre_start: the start date for pre fire imagery
    pre_end: the end date for pre fire imagery
    post_start: the start date for post fire imagery
    post_end: the end date for post_fire imagery
    geometry: the geometry to filter by
    """

    #landsat 5
    lt5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2').filterDate(pre_start, post_end).filterBounds(geometry)
    #landsat 7
    le7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2').filterDate(pre_start, post_end).filterBounds(geometry)
    #landsat 8
    lc8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate(pre_start, post_end).filterBounds(geometry)
    #sentinel 2
    sent = sent_2A.filterDate(pre_start, post_end).filterBounds(geometry)

    #select bands
    pre_lt5 = lt5.filterDate(pre_start, pre_end).map(mask).map(land_scale).select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']).map(to_float)

    #       #ensure we have imagery for the sensor
    if pre_lt5.size().getInfo() > 0 :


        #take the median
        pre_lt5 = pre_lt5.median().clip(geometry)

        #calculate nbr, ndvi and ndii
        pre_lt5_nbr = pre_lt5.normalizedDifference(['SR_B4', 'SR_B7']).select([0], ['NBR']).cast({'NBR': 'float'})
        pre_lt5_ndvi = pre_lt5.normalizedDifference(['SR_B4', 'SR_B3']).select([0], ['NDVI']).cast({'NDVI': 'float'})
        pre_lt5_ndii = pre_lt5.normalizedDifference(['SR_B4', 'SR_B5']).select([0], ['NDII']).cast({'NDII': 'float'})

        #add the bands back
        pre_lt5 = pre_lt5.addBands(pre_lt5_nbr).addBands(pre_lt5_ndvi).addBands(pre_lt5_ndii)

        #apply the corrections

        l5_pre_corr = pre_lt5.multiply(l5_corr[1]).add(pre_lt5.pow(2).multiply(l5_corr[2])).add(pre_lt5.pow(3).multiply(l5_corr[3])).add(l5_corr[0])

    #-------now do post-fire
    #select bands
    post_lt5 = lt5.filterDate(post_start, post_end).map(mask).map(land_scale).select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']).map(to_float)

    #       #ensure we have imagery for the sensor
    if post_lt5.size().getInfo() > 0 :



        #take the median
        post_lt5 = post_lt5.median().clip(geometry)

        #calculate nbr, ndvi and ndii
        post_lt5_nbr = post_lt5.normalizedDifference(['SR_B4', 'SR_B7']).select([0], ['NBR']).cast({'NBR': 'float'})
        post_lt5_ndvi = post_lt5.normalizedDifference(['SR_B4', 'SR_B3']).select([0], ['NDVI']).cast({'NDVI': 'float'})
        post_lt5_ndii = post_lt5.normalizedDifference(['SR_B4', 'SR_B5']).select([0], ['NDII']).cast({'NDII': 'float'})

        #add the bands back
        post_lt5 = post_lt5.addBands(post_lt5_nbr).addBands(post_lt5_ndvi).addBands(post_lt5_ndii)

        #apply the corrections

        l5_post_corr = post_lt5.multiply(l5_corr[1]).add(post_lt5.pow(2).multiply(l5_corr[2])).add(post_lt5.pow(3).multiply(l5_corr[3])).add(l5_corr[0])


      #         #------------------------------------------Landsat 7, no corrections but get things clipped and do pre fire/post_fire stuff


    #select bands
    pre_le7 = le7.filterDate(pre_start, pre_end).map(mask).map(land_scale).select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']).map(to_float)

    #       #ensure we have imagery for the sensor
    if pre_le7.size().getInfo() > 0 :



        #take the median
        pre_le7 = pre_le7.median().clip(geometry)

        #calculate nbr, ndvi and ndii
        pre_le7_nbr = pre_le7.normalizedDifference(['SR_B4', 'SR_B7']).select([0], ['NBR']).cast({'NBR': 'float'})
        pre_le7_ndvi = pre_le7.normalizedDifference(['SR_B4', 'SR_B3']).select([0], ['NDVI']).cast({'NDVI': 'float'})
        pre_le7_ndii = pre_le7.normalizedDifference(['SR_B4', 'SR_B5']).select([0], ['NDII']).cast({'NDII': 'float'})

        #add the bands back
        pre_le72 = pre_le7.addBands(pre_le7_nbr).addBands(pre_le7_ndvi).addBands(pre_le7_ndii)

    #-------now do post-fire
    #select bands
    post_le7 = le7.filterDate(post_start, post_end).map(mask).map(land_scale).select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']).map(to_float)
    #       #ensure we have imagery for the sensor
    if post_le7.size().getInfo() > 0 :


        #take the median
        post_le7 = post_le7.median().clip(geometry)

        #calculate nbr, ndvi and ndii
        post_le7_nbr = post_le7.normalizedDifference(['SR_B4', 'SR_B7']).select([0], ['NBR']).cast({'NBR': 'float'})
        post_le7_ndvi = post_le7.normalizedDifference(['SR_B4', 'SR_B3']).select([0], ['NDVI']).cast({'NDVI': 'float'})
        post_le7_ndii = post_le7.normalizedDifference(['SR_B4', 'SR_B5']).select([0], ['NDII']).cast({'NDII': 'float'})

        #add the bands back
        post_le72 = post_le7.addBands(post_le7_nbr).addBands(post_le7_ndvi).addBands(post_le7_ndii)

    #------------------------------------------Landsat 8 corrections


    #-------first do pre-fire

    #select bands
    pre_lc8 = lc8.filterDate(pre_start, pre_end).map(mask).map(land_scale).select(['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7'],['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']) .map(to_float)

    #       #ensure we have imagery for the sensor
    if pre_lc8.size().getInfo() > 0 :



        #take the median
        pre_lc8 = pre_lc8.median().clip(geometry)

        #calculate nbr, ndvi and ndii
        pre_lc8_nbr = pre_lc8.normalizedDifference(['SR_B4', 'SR_B7']).select([0], ['NBR']).cast({'NBR': 'float'})
        pre_lc8_ndvi = pre_lc8.normalizedDifference(['SR_B4', 'SR_B3']).select([0], ['NDVI']).cast({'NDVI': 'float'})
        pre_lc8_ndii = pre_lc8.normalizedDifference(['SR_B4', 'SR_B5']).select([0], ['NDII']).cast({'NDII': 'float'})

        #add the bands back
        pre_lc8 = pre_lc8.addBands(pre_lc8_nbr).addBands(pre_lc8_ndvi).addBands(pre_lc8_ndii)

        #apply the corrections

        l8_pre_corr = pre_lc8.multiply(l8_corr[1]).add(pre_lc8.pow(2).multiply(l8_corr[2])).add(pre_lc8.pow(3).multiply(l8_corr[3])).add(l8_corr[0])

    #-------now do post-fire
    #select bands
    post_lc8 = lc8.filterDate(post_start, post_end).map(mask).map(land_scale).select(['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7'],['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']) .map(to_float)

    #       #ensure we have imagery for the sensor
    if post_lc8.size().getInfo() > 0 :



        #take the median
        post_lc8 = post_lc8.median().clip(geometry)

        #calculate nbr, ndvi and ndii
        post_lc8_nbr = post_lc8.normalizedDifference(['SR_B4', 'SR_B7']).select([0], ['NBR']).cast({'NBR': 'float'})
        post_lc8_ndvi = post_lc8.normalizedDifference(['SR_B4', 'SR_B3']).select([0], ['NDVI']).cast({'NDVI': 'float'})
        post_lc8_ndii = post_lc8.normalizedDifference(['SR_B4', 'SR_B5']).select([0], ['NDII']).cast({'NDII': 'float'})

        #add the bands back
        post_lc8 = post_lc8.addBands(post_lc8_nbr).addBands(post_lc8_ndvi).addBands(post_lc8_ndii)

        #apply the corrections

        l8_post_corr = post_lc8.multiply(l8_corr[1]).add(post_lc8.pow(2).multiply(l8_corr[2])).add(post_lc8.pow(3).multiply(l8_corr[3])).add(l8_corr[0])

        # #          #------------------------------------------Sentinel 2 corrections, use landsat 8 coefficients


    ##-------first do pre-fire

    #select bands
    pre_sent = sent_2A.filterDate(pre_start, pre_end).map(sent_scale).select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']).map(to_float)

    #       #ensure we have imagery for the sensor
    if pre_sent.size().getInfo() > 0 :



          #take the median
        pre_sent = pre_sent.median().clip(geometry)

        #calculate nbr, ndvi and ndii
        pre_sent_nbr = pre_sent.normalizedDifference(['SR_B4', 'SR_B7']).select([0], ['NBR']).cast({'NBR': 'float'})
        pre_sent_ndvi = pre_sent.normalizedDifference(['SR_B4', 'SR_B3']).select([0], ['NDVI']).cast({'NDVI': 'float'})
        pre_sent_ndii = pre_sent.normalizedDifference(['SR_B4', 'SR_B5']).select([0], ['NDII']).cast({'NDII': 'float'})

        #add the bands back
        pre_sent = pre_sent.addBands(pre_sent_nbr).addBands(pre_sent_ndvi).addBands(pre_sent_ndii)

        #apply the corrections

        sent_pre_corr = pre_sent.multiply(l8_corr[1]).add(pre_sent.pow(2).multiply(l8_corr[2])).add(pre_sent.pow(3).multiply(l8_corr[3])).add(l8_corr[0])

    #-------now do post-fire
    #select bands
    post_sent = sent_2A.filterDate(post_start, post_end).map(sent_scale).select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']).map(to_float)

    #       #ensure we have imagery for the sensor
    if post_sent.size().getInfo() > 0 :



        #take the median
        post_sent = post_sent.median().clip(geometry)

        #calculate nbr, ndvi and ndii
        post_sent_nbr = post_sent.normalizedDifference(['SR_B4', 'SR_B7']).select([0], ['NBR']).cast({'NBR': 'float'})
        post_sent_ndvi = post_sent.normalizedDifference(['SR_B4', 'SR_B3']).select([0], ['NDVI']).cast({'NDVI': 'float'})
        post_sent_ndii = post_sent.normalizedDifference(['SR_B4', 'SR_B5']).select([0], ['NDII']).cast({'NDII': 'float'})

        #add the bands back
        post_sent = post_sent.addBands(post_sent_nbr).addBands(post_sent_ndvi).addBands(post_sent_ndii)

        #apply the corrections

        sent_post_corr = post_sent.multiply(l8_corr[1]).add(post_sent.pow(2).multiply(l8_corr[2])).add(post_sent.pow(3).multiply(l8_corr[3])).add(l8_corr[0])


    #try to see if image exists, if so append

    #----------------------all prefire

    #       #empty list for pre-fire, use this to combine if we have land 5, 7, 8 or sentinel
    pre_input = []

    try:
        l5_pre_corr.getInfo()
        pre_input.append(l5_pre_corr)

    except:
        pass

    try:
        pre_le72.getInfo()
        pre_input.append(pre_le72)

    except:
        pass

    try:
        l8_pre_corr.getInfo()
        pre_input.append(l8_pre_corr)

    except:
        pass

    try:
        sent_pre_corr.getInfo()
        pre_input.append(sent_pre_corr)

    except:
        pass


    #----------------------all postfire

    #         #       #empty list for post-fire, use this to combine if we have land 5, 7, 8 or sentinel
    post_input = []

    try:
        l5_post_corr.getInfo()
        post_input.append(l5_post_corr)

    except:
        pass

    try:
        post_le72.getInfo()
        post_input.append(post_le72)

    except:
        pass

    try:
        l8_post_corr.getInfo()
        post_input.append(l8_post_corr)

    except:
        pass

    try:
        sent_post_corr.getInfo()
        post_input.append(sent_post_corr)

    except:
        pass

    #return the two lists of pre input and post input
    return pre_input, post_input



In [21]:
all_territories = ee.List(input_grid.distinct(["Id"]).aggregate_array("Id"))
all_territories = all_territories.getInfo()

years = [2015]
for year in years:
    
    for territory in all_territories:
        
    
        territory22 = input_grid.filter(ee.Filter.eq('Id',  territory))
        
        #create output image collection
        # base = 'projects/gee-serdp-upload/assets/cnn_mapping/nwt_nbac_2d_test2'
        base = 'users/spotter/fire_cnn/Russia/post_nbr_counts/' + str(year) + '_0401_0501'

        !earthengine create collection $base
        
        #full pathway to images within the collection
        full_out =os.path.join(base, str(year) + '_' + str(territory))
        
        #check if the asset exists, if it does skip it
        try:
            exists = ee.data.getAsset(full_out)
  
            print(f"Image {full_out} exists")

        except:
            print(f"Image {full_out} does not exist")
            
            #get pre dates
            pre_start = str(year - 1) + '-04-01'
            pre_end = str(year - 1) + '-05-01'

            #get post dates


            post_start = str(year + 1) + '-04-01'
            post_end = str(year + 1) + '-05-01'


            startYear = str(year - 1)
            endYear = str(year + 1)
            startDay  = '01-01' # what is the beginning of date filter | month-day
            endDay     = '12-30' # what is the end of date filter | month-day

            #########################################################################################################
            ###### ANNUAL SR TIME SERIES COLLECTION BUILDING FUNCTIONS ##### 
            #########################################################################################################

            #------ RETRIEVE A SENSOR SR COLLECTION FUNCTION -----
            def getSRcollection(start_date, end_date, sensor):
                # get a landsat collection for given year, day range, and sensor
                srCollection = ee.ImageCollection('LANDSAT/'+ sensor + '/C02/T1_L2').filterDate(start_date, end_date)

                return srCollection

            #get all collection5
            # lt4 = getSRcollection(startYear+'-'+startDay, endYear+'-'+endDay, 'LT04')  
            lt5 = getSRcollection(startYear+'-'+startDay, endYear+'-'+endDay, 'LT05').filterBounds(territory22)    
            le7 = getSRcollection(startYear+'-'+startDay, endYear+'-'+endDay, 'LE07').filterBounds(territory22)     
            lc8 = getSRcollection(startYear+'-'+startDay, endYear+'-'+endDay, 'LC08').filterBounds(territory22)        
            sent= sent_2A.filterDate(startYear+'-'+startDay, endYear+'-'+endDay).filterBounds(territory22)

            #         #------------------------------------------Landsat 5 corrections

            #select bands
            pre_lt5 = lt5.filterDate(pre_start, pre_end).map(mask).map(land_scale).select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']).map(to_float)

            #       #ensure we have imagery for the sensor
            if pre_lt5.size().getInfo() > 0 :


                #take the median
                pre_lt5 = pre_lt5.median().clip(territory22)

                #calculate nbr, ndvi and ndii
                pre_lt5_nbr = pre_lt5.normalizedDifference(['SR_B4', 'SR_B7']).select([0], ['NBR']).cast({'NBR': 'float'})
                pre_lt5_ndvi = pre_lt5.normalizedDifference(['SR_B4', 'SR_B3']).select([0], ['NDVI']).cast({'NDVI': 'float'})
                pre_lt5_ndii = pre_lt5.normalizedDifference(['SR_B4', 'SR_B5']).select([0], ['NDII']).cast({'NDII': 'float'})

                #add the bands back
                pre_lt5 = pre_lt5.addBands(pre_lt5_nbr).addBands(pre_lt5_ndvi).addBands(pre_lt5_ndii)

                #apply the corrections

                l5_pre_corr = pre_lt5.multiply(l5_corr[1]).add(pre_lt5.pow(2).multiply(l5_corr[2])).add(pre_lt5.pow(3).multiply(l5_corr[3])).add(l5_corr[0])

            #-------now do post-fire
            #select bands
            post_lt5 = lt5.filterDate(post_start, post_end).map(mask).map(land_scale).select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']).map(to_float)

            #       #ensure we have imagery for the sensor
            if post_lt5.size().getInfo() > 0 :



                #take the median
                post_lt5 = post_lt5.median().clip(territory22)

                #calculate nbr, ndvi and ndii
                post_lt5_nbr = post_lt5.normalizedDifference(['SR_B4', 'SR_B7']).select([0], ['NBR']).cast({'NBR': 'float'})
                post_lt5_ndvi = post_lt5.normalizedDifference(['SR_B4', 'SR_B3']).select([0], ['NDVI']).cast({'NDVI': 'float'})
                post_lt5_ndii = post_lt5.normalizedDifference(['SR_B4', 'SR_B5']).select([0], ['NDII']).cast({'NDII': 'float'})

                #add the bands back
                post_lt5 = post_lt5.addBands(post_lt5_nbr).addBands(post_lt5_ndvi).addBands(post_lt5_ndii)

                #apply the corrections

                l5_post_corr = post_lt5.multiply(l5_corr[1]).add(post_lt5.pow(2).multiply(l5_corr[2])).add(post_lt5.pow(3).multiply(l5_corr[3])).add(l5_corr[0])


                #         #------------------------------------------Landsat 7, no corrections but get things clipped and do pre fire/post_fire stuff


            #select bands
            pre_le7 = le7.filterDate(pre_start, pre_end).map(mask).map(land_scale).select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']).map(to_float)

            #       #ensure we have imagery for the sensor
            if pre_le7.size().getInfo() > 0 :



                #take the median
                pre_le7 = pre_le7.median().clip(territory22)

                #calculate nbr, ndvi and ndii
                pre_le7_nbr = pre_le7.normalizedDifference(['SR_B4', 'SR_B7']).select([0], ['NBR']).cast({'NBR': 'float'})
                pre_le7_ndvi = pre_le7.normalizedDifference(['SR_B4', 'SR_B3']).select([0], ['NDVI']).cast({'NDVI': 'float'})
                pre_le7_ndii = pre_le7.normalizedDifference(['SR_B4', 'SR_B5']).select([0], ['NDII']).cast({'NDII': 'float'})

                #add the bands back
                pre_le72 = pre_le7.addBands(pre_le7_nbr).addBands(pre_le7_ndvi).addBands(pre_le7_ndii)

            #-------now do post-fire
            #select bands
            post_le7 = le7.filterDate(post_start, post_end).map(mask).map(land_scale).select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']).map(to_float)
            #       #ensure we have imagery for the sensor
            if post_le7.size().getInfo() > 0 :


                #take the median
                post_le7 = post_le7.median().clip(territory22)

                #calculate nbr, ndvi and ndii
                post_le7_nbr = post_le7.normalizedDifference(['SR_B4', 'SR_B7']).select([0], ['NBR']).cast({'NBR': 'float'})
                post_le7_ndvi = post_le7.normalizedDifference(['SR_B4', 'SR_B3']).select([0], ['NDVI']).cast({'NDVI': 'float'})
                post_le7_ndii = post_le7.normalizedDifference(['SR_B4', 'SR_B5']).select([0], ['NDII']).cast({'NDII': 'float'})

                #add the bands back
                post_le72 = post_le7.addBands(post_le7_nbr).addBands(post_le7_ndvi).addBands(post_le7_ndii)

            #------------------------------------------Landsat 8 corrections


            #-------first do pre-fire

            #select bands
            pre_lc8 = lc8.filterDate(pre_start, pre_end).map(mask).map(land_scale).select(['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7'],['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']) .map(to_float)

            #       #ensure we have imagery for the sensor
            if pre_lc8.size().getInfo() > 0 :



                #take the median
                pre_lc8 = pre_lc8.median().clip(territory22)

                #calculate nbr, ndvi and ndii
                pre_lc8_nbr = pre_lc8.normalizedDifference(['SR_B4', 'SR_B7']).select([0], ['NBR']).cast({'NBR': 'float'})
                pre_lc8_ndvi = pre_lc8.normalizedDifference(['SR_B4', 'SR_B3']).select([0], ['NDVI']).cast({'NDVI': 'float'})
                pre_lc8_ndii = pre_lc8.normalizedDifference(['SR_B4', 'SR_B5']).select([0], ['NDII']).cast({'NDII': 'float'})

                #add the bands back
                pre_lc8 = pre_lc8.addBands(pre_lc8_nbr).addBands(pre_lc8_ndvi).addBands(pre_lc8_ndii)

                #apply the corrections

                l8_pre_corr = pre_lc8.multiply(l8_corr[1]).add(pre_lc8.pow(2).multiply(l8_corr[2])).add(pre_lc8.pow(3).multiply(l8_corr[3])).add(l8_corr[0])

            #-------now do post-fire
              #select bands
            post_lc8 = lc8.filterDate(post_start, post_end).map(mask).map(land_scale).select(['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7'],['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']) .map(to_float)

            #       #ensure we have imagery for the sensor
            if post_lc8.size().getInfo() > 0 :



            #take the median
                post_lc8 = post_lc8.median().clip(territory22)

                #calculate nbr, ndvi and ndii
                post_lc8_nbr = post_lc8.normalizedDifference(['SR_B4', 'SR_B7']).select([0], ['NBR']).cast({'NBR': 'float'})
                post_lc8_ndvi = post_lc8.normalizedDifference(['SR_B4', 'SR_B3']).select([0], ['NDVI']).cast({'NDVI': 'float'})
                post_lc8_ndii = post_lc8.normalizedDifference(['SR_B4', 'SR_B5']).select([0], ['NDII']).cast({'NDII': 'float'})

                #add the bands back
                post_lc8 = post_lc8.addBands(post_lc8_nbr).addBands(post_lc8_ndvi).addBands(post_lc8_ndii)

                #apply the corrections

                l8_post_corr = post_lc8.multiply(l8_corr[1]).add(post_lc8.pow(2).multiply(l8_corr[2])).add(post_lc8.pow(3).multiply(l8_corr[3])).add(l8_corr[0])

                # #          #------------------------------------------Sentinel 2 corrections, use landsat 8 coefficients


            ##-------first do pre-fire

            #select bands
            pre_sent = sent_2A.filterDate(pre_start, pre_end).map(sent_scale).select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']).map(to_float)

            #       #ensure we have imagery for the sensor
            if pre_sent.size().getInfo() > 0 :



                    #take the median
                pre_sent = pre_sent.median().clip(territory22)

                #calculate nbr, ndvi and ndii
                pre_sent_nbr = pre_sent.normalizedDifference(['SR_B4', 'SR_B7']).select([0], ['NBR']).cast({'NBR': 'float'})
                pre_sent_ndvi = pre_sent.normalizedDifference(['SR_B4', 'SR_B3']).select([0], ['NDVI']).cast({'NDVI': 'float'})
                pre_sent_ndii = pre_sent.normalizedDifference(['SR_B4', 'SR_B5']).select([0], ['NDII']).cast({'NDII': 'float'})

                #add the bands back
                pre_sent = pre_sent.addBands(pre_sent_nbr).addBands(pre_sent_ndvi).addBands(pre_sent_ndii)

                #apply the corrections

                sent_pre_corr = pre_sent.multiply(l8_corr[1]).add(pre_sent.pow(2).multiply(l8_corr[2])).add(pre_sent.pow(3).multiply(l8_corr[3])).add(l8_corr[0])

            #-------now do post-fire
            #select bands
            post_sent = sent_2A.filterDate(post_start, post_end).map(sent_scale).select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']).map(to_float)

            #       #ensure we have imagery for the sensor
            if post_sent.size().getInfo() > 0 :



                #take the median
                post_sent = post_sent.median().clip(territory22)

                #calculate nbr, ndvi and ndii
                post_sent_nbr = post_sent.normalizedDifference(['SR_B4', 'SR_B7']).select([0], ['NBR']).cast({'NBR': 'float'})
                post_sent_ndvi = post_sent.normalizedDifference(['SR_B4', 'SR_B3']).select([0], ['NDVI']).cast({'NDVI': 'float'})
                post_sent_ndii = post_sent.normalizedDifference(['SR_B4', 'SR_B5']).select([0], ['NDII']).cast({'NDII': 'float'})

                #add the bands back
                post_sent = post_sent.addBands(post_sent_nbr).addBands(post_sent_ndvi).addBands(post_sent_ndii)

                #apply the corrections

                sent_post_corr = post_sent.multiply(l8_corr[1]).add(post_sent.pow(2).multiply(l8_corr[2])).add(post_sent.pow(3).multiply(l8_corr[3])).add(l8_corr[0])


            #try to see if image exists, if so append

            #----------------------all prefire

            #       #empty list for pre-fire, use this to combine if we have land 5, 7, 8 or sentinel 
            pre_input = []

            try:
                l5_pre_corr.getInfo()
                pre_input.append(l5_pre_corr)

            except:
                pass

            try:
                pre_le72.getInfo()
                pre_input.append(pre_le72)

            except:
                pass

            try:
                l8_pre_corr.getInfo()
                pre_input.append(l8_pre_corr)

            except:
                pass

            try:
                sent_pre_corr.getInfo()
                pre_input.append(sent_pre_corr)

            except:
                pass


            #----------------------all postfire

            #         #       #empty list for post-fire, use this to combine if we have land 5, 7, 8 or sentinel 
            post_input = []

            try:
                l5_post_corr.getInfo()
                post_input.append(l5_post_corr)

            except:
                pass

            try:
                post_le72.getInfo()
                post_input.append(post_le72)

            except:
                pass

            try:
                l8_post_corr.getInfo()
                post_input.append(l8_post_corr)

            except:
                pass

            try:
                sent_post_corr.getInfo()
                post_input.append(sent_post_corr)

            except:
                pass

            if (len(pre_input) >0) and (len(post_input) > 0):

                #take the median of the image collections
                pre_input = ee.ImageCollection(pre_input)
                post_input = ee.ImageCollection(post_input)



              #try and except,, if we have data in both pre and post get the dnbr, otherwise pass

                # try:
                #     if (pre_input.size().getInfo() > 0) and (post_input.size().getInfo() > 0):

                pre_input = pre_input.median()
                post_input= post_input.median()

                #difference the bands
                raw_bands = pre_input.subtract(post_input).multiply(1000)

                # b1 = raw_bands.select('SR_B1').cast({'SR_B1':'short'}) #0
                # b2 = raw_bands.select('SR_B2').cast({'SR_B2':'short'}) #1
                # b3 = raw_bands.select('SR_B3').cast({'SR_B3':'short'}) #2
                # b4 = raw_bands.select('SR_B4').cast({'SR_B4':'short'}) #3
                # b5 = raw_bands.select('SR_B5').cast({'SR_B5':'short'}) #4
                # b6 = raw_bands.select('SR_B7').cast({'SR_B7':'short'}) #5
                # b7 = raw_bands.select('NBR').cast({'NBR':'short'}) #band 6 is dnbr is numpy
                # b8 = raw_bands.select('NDVI').cast({'NDVI':'short'}) #7
                # b9 = raw_bands.select('NDII').cast({'NDII':'short'}) #8
                # b10 = raw_bands.select('y').cast({'y':'float'})
                # b10 = y.select('y').cast({'y':'short'})

                b1 = raw_bands.select('SR_B1').cast({'SR_B1':'short'}) #0
                b2 = raw_bands.select('SR_B2').cast({'SR_B2':'short'}) #1
                b3 = raw_bands.select('SR_B3').cast({'SR_B3':'short'}) #2
                b4 = raw_bands.select('SR_B4').cast({'SR_B4':'short'}) #3
                b5 = raw_bands.select('SR_B5').cast({'SR_B5':'short'}) #4
                b6 = raw_bands.select('SR_B7').cast({'SR_B7':'short'}) #5
                b7 = raw_bands.select('NBR').cast({'NBR':'short'}) #band 6 is dnbr is numpy
                b8 = raw_bands.select('NDVI').cast({'NDVI':'short'}) #7
                b9 = raw_bands.select('NDII').cast({'NDII':'short'}) #8


                #if using all bands
                raw_bands = b1.addBands(b2).addBands(b3).addBands(b4).addBands(b5).addBands(b6).addBands(b7).addBands(b8).addBands(b9)

                #if not using all bands
                # raw_bands = b7.addBands(b8).addBands(b9)

        #         raw_bands = raw_bands.clip(territory22)

        #         #normalize here
        #           #normalize 0-1
        #         #calculate the min and max value of an image
        #         minMax = raw_bands.reduceRegion(
        #         reducer = ee.Reducer.minMax(),
        #         geometry = territory22,
        #         scale = 30,
        #         maxPixels = 10e9,
        #         )

        #         b1 = raw_bands.select('SR_B1').unitScale(minMax.get('SR_B1_min'), minMax.get('SR_B1_max'))
        #         b2 = raw_bands.select('SR_B2').toFloat().unitScale(minMax.get('SR_B2_min'), minMax.get('SR_B2_max')).toFloat()
        #         b3 = raw_bands.select('SR_B3').toFloat().unitScale(minMax.get('SR_B3_min'), minMax.get('SR_B3_max')).toFloat()
        #         b4 = raw_bands.select('SR_B4').toFloat().unitScale(minMax.get('SR_B4_min'), minMax.get('SR_B4_max')).toFloat()
        #         b5 = raw_bands.select('SR_B5').toFloat().unitScale(minMax.get('SR_B5_min'), minMax.get('SR_B5_max')).toFloat()
        #         b6 = raw_bands.select('SR_B7').toFloat().unitScale(minMax.get('SR_B7_min'), minMax.get('SR_B7_max')).toFloat()
        #         b7 = raw_bands.select('NBR').toFloat().unitScale(minMax.get('NBR_min'), minMax.get('NBR_max')).toFloat()
        #         b8 = raw_bands.select('NDVI').toFloat().unitScale(minMax.get('NDVI_min'), minMax.get('NDVI_max')).toFloat()
        #         b9 = raw_bands.select('NDII').toFloat().unitScale(minMax.get('NDII_min'), minMax.get('NDII_max')).toFloat()

        #         raw_bands = b1.addBands(b2).addBands(b3).addBands(b4).addBands(b5).addBands(b6).addBands(b7).addBands(b8).addBands(b9)

                #loop through each grid cell
                storage_client = storage.Client()

                # for id in np.arange(0, grd.size().getInfo() + 1, 1):


                raw_bands2 = raw_bands.clip(territory22)

                #normalize 0-1
        #                 #calculate the min and max value of an image
        #                 minMax = raw_bands2.reduceRegion(
        #                 reducer = ee.Reducer.minMax(),
        #                 geometry = territory22.geometry(),
        #                 scale = 30,
        #                 maxPixels = 10e9,
        #                 )

        #                 b1 = raw_bands2.select('SR_B1').unitScale(minMax.get('SR_B1_min'), minMax.get('SR_B1_max'))
        #                 b2 = raw_bands2.select('SR_B2').toFloat().unitScale(minMax.get('SR_B2_min'), minMax.get('SR_B2_max')).toFloat()
        #                 b3 = raw_bands2.select('SR_B3').toFloat().unitScale(minMax.get('SR_B3_min'), minMax.get('SR_B3_max')).toFloat()
        #                 b4 = raw_bands2.select('SR_B4').toFloat().unitScale(minMax.get('SR_B4_min'), minMax.get('SR_B4_max')).toFloat()
        #                 b5 = raw_bands2.select('SR_B5').toFloat().unitScale(minMax.get('SR_B5_min'), minMax.get('SR_B5_max')).toFloat()
        #                 b6 = raw_bands2.select('SR_B7').toFloat().unitScale(minMax.get('SR_B7_min'), minMax.get('SR_B7_max')).toFloat()
        #                 b7 = raw_bands2.select('NBR').toFloat().unitScale(minMax.get('NBR_min'), minMax.get('NBR_max')).toFloat()
        #                 b8 = raw_bands2.select('NDVI').toFloat().unitScale(minMax.get('NDVI_min'), minMax.get('NDVI_max')).toFloat()
        #                 b9 = raw_bands2.select('NDII').toFloat().unitScale(minMax.get('NDII_min'), minMax.get('NDII_max')).toFloat()





#                 b1_min = glob_norm.reset_index().query('index == 0')['0'].values[0]
#                 b1_max = glob_norm.reset_index().query('index == 1')['0'].values[0]

#                 b2_min = glob_norm.reset_index().query('index == 0')['1'].values[0]
#                 b2_max = glob_norm.reset_index().query('index == 1')['1'].values[0]

#                 b3_min = glob_norm.reset_index().query('index == 0')['2'].values[0]
#                 b3_max = glob_norm.reset_index().query('index == 1')['2'].values[0]

#                 b4_min = glob_norm.reset_index().query('index == 0')['3'].values[0]
#                 b4_max = glob_norm.reset_index().query('index == 1')['3'].values[0]

#                 b5_min = glob_norm.reset_index().query('index == 0')['4'].values[0]
#                 b5_max = glob_norm.reset_index().query('index == 1')['4'].values[0]

#                 b6_min = glob_norm.reset_index().query('index == 0')['5'].values[0]
#                 b6_max = glob_norm.reset_index().query('index == 1')['5'].values[0]

#                 b7_min = glob_norm.reset_index().query('index == 0')['6'].values[0]
#                 b7_max = glob_norm.reset_index().query('index == 1')['6'].values[0]

#                 b8_min = glob_norm.reset_index().query('index == 0')['7'].values[0]
#                 b8_max = glob_norm.reset_index().query('index == 1')['7'].values[0]

#                 b9_min = glob_norm.reset_index().query('index == 0')['8'].values[0]
#                 b9_max = glob_norm.reset_index().query('index == 1')['8'].values[0]

#                 b7_min = glob_norm.reset_index().query('index == 0')['6'].values[0]
#                 b7_max = glob_norm.reset_index().query('index == 1')['6'].values[0]

#                 b8_min = glob_norm.reset_index().query('index == 0')['7'].values[0]
#                 b8_max = glob_norm.reset_index().query('index == 1')['7'].values[0]

#                 b9_min = glob_norm.reset_index().query('index == 0')['8'].values[0]
#                 b9_max = glob_norm.reset_index().query('index == 1')['8'].values[0]



#                 b1 = raw_bands2.select('SR_B1').toFloat().unitScale(b1_min, b1_max)
#                 b2 = raw_bands2.select('SR_B2').toFloat().unitScale(b2_min, b2_max)#.toFloat()
#                 b3 = raw_bands2.select('SR_B3').toFloat().unitScale(b3_min, b3_max)#.toFloat()
#                 b4 = raw_bands2.select('SR_B4').toFloat().unitScale(b4_min, b4_max)#.toFloat()
#                 b5 = raw_bands2.select('SR_B5').toFloat().unitScale(b5_min, b5_max)#.toFloat()
#                 b6 = raw_bands2.select('SR_B7').toFloat().unitScale(b6_min, b6_max)#.toFloat()
#                 b7 = raw_bands2.select('NBR').toFloat().unitScale(b7_min, b7_max)#.toFloat()
#                 b8 = raw_bands2.select('NDVI').toFloat().unitScale(b8_min, b8_max)#.toFloat()
#                 b9 = raw_bands2.select('NDII').toFloat().unitScale(b9_min, b9_max)#.toFloat()





                # for_predict = b1.addBands(b2).addBands(b3).addBands(b4).addBands(b5).addBands(b6).addBands(b7).addBands(b8).addBands(b9)
                # for_predict = b7.addBands(b8).addBands(b9)
                for_predict = raw_bands2.select('NBR')
#                 for_predict = b7


#                 # for_predict = raw_bands.clip(territory22)

#                 for_predict = for_predict.reproject(crs = 'EPSG:3413', scale = 30)

#                 arrayImage = for_predict.float().toArray().rename('array')

#                 # #predict model
#                 predictions = model.predictImage(arrayImage).arrayGet([0])

#         #                 predictions2 = predictions.updateMask(predictions.gt(0.5))
#         #                 predictions2 = predictions2.where(predictions2.gte(0.5), 1)

#                 # final = b7.addBands(b8).addBands(b9).addBands(predictions).addBands(predictions2)
#                 final = predictions
#                 final = final.set('system:time_start', str(year) + '-01-01')

                #denoise


                # final = b1.addBands(b2).addBands(b3).addBands(b4).addBands(b5).addBands(b6).addBands(b7).addBands(b8).addBands(b9).addBands(predictions).addBands(predictions2)

                print(f"Downloading {year} and {territory}")

        #                 task = ee.batch.Export.image.toCloudStorage(
        #                                         image = final,
        #                                         region=territory22.geometry(), 
        #                                         description='predicted_' + str(year) + '_' + str(i),
        #                                         scale=30,
        #                                         crs='EPSG:3413',
        #                                         maxPixels=1e13,
        #                                         bucket = 'smp-scratch')

                #                 task.start()

                # base = f"users/spotter/fire_cnn/mtbs_predictions/all_ak_ca_dnbr_l8_crop_mtn_small"
                # !earthengine create collection base
                task = ee.batch.Export.image.toAsset(
                                  # image = for_predict.multiply(1000).toShort(),
                                  image = for_predict.toShort(),

                                  region=territory22.geometry(), 
                                  description= str(year) + '_' + str(territory),
                                  scale=30,
                                  crs='EPSG:3413',
                                  # crs= 'EPSG:4326',
                                  maxPixels=1e13,
                                  assetId =  full_out)
                                  # assetId =  'users/spotter/fire_cnn/mtbs_predictions/' + str(year) + '_' + str(territory22))


                task.start()



                        # except:
                        #     pass

Asset users/spotter/fire_cnn/Russia/dnbr_check/2015_0501_0831 already exists.
Image users/spotter/fire_cnn/Russia/dnbr_check/2015_0501_0831/2015_0 exists
Asset users/spotter/fire_cnn/Russia/dnbr_check/2015_0501_0831 already exists.
Image users/spotter/fire_cnn/Russia/dnbr_check/2015_0501_0831/2015_1 exists
Asset users/spotter/fire_cnn/Russia/dnbr_check/2015_0501_0831 already exists.
Image users/spotter/fire_cnn/Russia/dnbr_check/2015_0501_0831/2015_2 exists
Asset users/spotter/fire_cnn/Russia/dnbr_check/2015_0501_0831 already exists.
Image users/spotter/fire_cnn/Russia/dnbr_check/2015_0501_0831/2015_3 exists
Asset users/spotter/fire_cnn/Russia/dnbr_check/2015_0501_0831 already exists.
Image users/spotter/fire_cnn/Russia/dnbr_check/2015_0501_0831/2015_4 exists
Asset users/spotter/fire_cnn/Russia/dnbr_check/2015_0501_0831 already exists.
Image users/spotter/fire_cnn/Russia/dnbr_check/2015_0501_0831/2015_5 exists
Asset users/spotter/fire_cnn/Russia/dnbr_check/2015_0501_0831 already exists